# Web scraping

[![Open In Collab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/langchain-ai/langchain/blob/master/docs/extras/use_cases/web_scraping.ipynb)

## Use case

[Web research](https://blog.langchain.dev/automating-web-research/) is one of the killer LLM applications:

* Users have [highlighted it](https://twitter.com/GregKamradt/status/1679913813297225729?s=20) as one of his top desired AI tools. 
* OSS repos like [gpt-researcher](https://github.com/assafelovic/gpt-researcher) are growing in popularity. 
 
![Image description](/img/web_scraping.png)
 
## Overview

Gathering content from the web has a few components:

* `Search`: Query to url (e.g., using `GoogleSearchAPIWrapper`).
* `Loading`: Url to HTML  (e.g., using `AsyncHtmlLoader`, `AsyncChromiumLoader`, etc).
* `Transforming`: HTML to formatted text (e.g., using `HTML2Text` or `Beautiful Soup`).

## Quickstart

In [ ]:
! pip install -q openai langchain playwright beautifulsoup4
! playwright install

# Set env var OPENAI_API_KEY or load from a .env file:
# import dotenv
# dotenv.load_env()

Scraping HTML content using a headless instance of Chromium.

* The async nature of the scraping process is handled using Python's asyncio library.
* The actual interaction with the web pages is handled by Playwright.

In [14]:
from langchain.document_loaders import AsyncChromiumLoader
from langchain.document_transformers import BeautifulSoupTransformer

# Load HTML
loader = AsyncChromiumLoader(["https://www.wsj.com"])
html = loader.load()

Scrape text content within `<p>, <li>, <div>, and <a>` tags from the HTML content:

* `<p>`: The paragraph tag. It defines a paragraph in HTML and is used to group together related sentences and/or phrases.
 
* `<li>`: The list item tag. It is used within ordered (`<ol>`) and unordered (`<ul>`) lists to define individual items within the list.
 
* `<div>`: The division tag. It is a block-level element used to group other inline or block-level elements.
 
* `<a>`: The anchor tag. It is used to define hyperlinks.

In the case of Wall Street Journal (or a lot of news websites, like CNN), their news headlines and summaries are all in `<span>` tags, so for best performance, we target only this tag. Other websites might be different.

In [16]:
# Transform
bs_transformer = BeautifulSoupTransformer()
docs_transformed = bs_transformer.transform_documents(html,tags_to_extract=["span"])

In [17]:
# Result
docs_transformed[0].page_content[0:500]

'Conservative legal activists are challenging Amazon, Comcast and others using many of the same tools that helped kill affirmative-action programs in colleges.1,1259 min read U.S. stock indexes fell and government-bond prices climbed, after Moody’s lowered credit ratings for 10 smaller U.S. banks and said it was reviewing ratings for six larger ones. The Dow industrials dropped more than 150 points.3 min read Penn Entertainment’s Barstool Sportsbook app will be rebranded as ESPN Bet this fall as '

These `Documents` now are staged for downstream usage in various LLM apps, as discussed below.

## Loader

### AsyncHtmlLoader

The [AsyncHtmlLoader](docs/integrations/document_loaders/async_html) uses the `aiohttp` library to make asynchronous HTTP requests, suitable for simpler and lightweight scraping.

### AsyncChromiumLoader

The [AsyncChromiumLoader](docs/integrations/document_loaders/async_chromium) uses Playwright to launch a Chromium instance, which can handle JavaScript rendering and more complex web interactions.

Chromium is one of the browsers supported by Playwright, a library used to control browser automation. 

Headless mode means that the browser is running without a graphical user interface, which is commonly used for web scrapin.

In [ ]:
from langchain.document_loaders import AsyncHtmlLoader
urls = ["https://www.espn.com","https://lilianweng.github.io/posts/2023-06-23-agent/"]
loader = AsyncHtmlLoader(urls)
docs = loader.load()

## Transformer

### HTML2Text

[HTML2Text](docs/integrations/document_transformers/html2text) provides a straightforward conversion of HTML content into plain text (with markdown-like formatting) without any specific tag manipulation. 

It's best suited for scenarios where the goal is to extract human-readable text without needing to manipulate specific HTML elements.

### Beautiful Soup
 
Beautiful Soup offers more fine-grained control over HTML content, enabling specific tag extraction, removal, and content cleaning. 

It's suited for cases where you want to extract specific information and clean up the HTML content according to your needs.

In [5]:
from langchain.document_loaders import AsyncHtmlLoader
urls = ["https://www.espn.com", "https://lilianweng.github.io/posts/2023-06-23-agent/"]
loader = AsyncHtmlLoader(urls)
docs = loader.load()

Fetching pages: 100%|#############################################################################################################| 2/2 [00:00<00:00,  7.01it/s]


In [7]:
from langchain.document_transformers import Html2TextTransformer
html2text = Html2TextTransformer()
docs_transformed = html2text.transform_documents(docs)
docs_transformed[0].page_content[0:500]

"Skip to main content  Skip to navigation\n\n<\n\n>\n\nMenu\n\n## ESPN\n\n  * Search\n\n  *   * scores\n\n  * NFL\n  * MLB\n  * NBA\n  * NHL\n  * Soccer\n  * NCAAF\n  * …\n\n    * Women's World Cup\n    * LLWS\n    * NCAAM\n    * NCAAW\n    * Sports Betting\n    * Boxing\n    * CFL\n    * NCAA\n    * Cricket\n    * F1\n    * Golf\n    * Horse\n    * MMA\n    * NASCAR\n    * NBA G League\n    * Olympic Sports\n    * PLL\n    * Racing\n    * RN BB\n    * RN FB\n    * Rugby\n    * Tennis\n    * WNBA\n    * WWE\n    * X Games\n    * XFL\n\n  * More"

## Scraping with extraction

### LLM with function calling

Web scraping is challenging for many reasons. 

One of them is the changing nature of modern websites' layouts and content, which requires modifying scraping scripts to accommodate the changes.

Using Function (e.g., OpenAI) with an extraction chain, we avoid having to change your code constantly when websites change. 

We're using `gpt-3.5-turbo-0613` to guarantee access to OpenAI Functions feature (although this might be available to everyone by time of writing). 

We're also keeping `temperature` at `0` to keep randomness of the LLM down.

In [9]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613")

### Define a schema

Next, you define a schema to specify what kind of data you want to extract. 

Here, the key names matter as they tell the LLM what kind of information they want. 

So, be as detailed as possible. 

In this example, we want to scrape only news article's name and summary from The Wall Street Journal website.

In [10]:
from langchain.chains import create_extraction_chain

schema = {
    "properties": {
        "news_article_title": {"type": "string"},
        "news_article_summary": {"type": "string"},
    },
    "required": ["news_article_title", "news_article_summary"],
}

def extract(content: str, schema: dict):
    return create_extraction_chain(schema=schema, llm=llm).run(content)

### Run the web scraper w/ BeautifulSoup

As shown above, we'll using `BeautifulSoupTransformer`.

In [39]:
import pprint
from langchain.text_splitter import RecursiveCharacterTextSplitter

def scrape_with_playwright(urls, schema):
    
    loader = AsyncChromiumLoader(urls)
    docs = loader.load()
    bs_transformer = BeautifulSoupTransformer()
    docs_transformed = bs_transformer.transform_documents(docs)
    print("Extracting content with LLM")
    
    # Grab the first 1000 tokens of the site
    splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=1000, 
                                                                    chunk_overlap=0)
    splits = splitter.split_documents(docs_transformed)
    
    # Process the first split 
    extracted_content = extract(
        schema=schema, content=splits[0].page_content
    )
    pprint.pprint(extracted_content)
    return extracted_content

urls = ["https://www.wsj.com"]
extracted_content = scrape_with_playwright(urls, schema=schema)

Extracting content with LLM
[{'news_article_summary': 'Conservative legal activists are challenging '
                          'Amazon, Comcast and others using many of the same '
                          'tools that helped kill affirmative-action programs '
                          'in colleges.',
  'news_article_title': 'Conservative legal activists challenge Amazon and '
                        'Comcast'},
 {'news_article_summary': 'U.S. stock indexes fell and government-bond prices '
                          'climbed, after Moody’s lowered credit ratings for '
                          '10 smaller U.S. banks and said it was reviewing '
                          'ratings for six larger ones.',
  'news_article_title': "Moody's lowers credit ratings for US banks"},
 {'news_article_summary': 'Penn Entertainment’s Barstool Sportsbook app will '
                          'be rebranded as ESPN Bet this fall as part of a '
                          '10-year agreement.',
  'news_article

Looking at the [LangSmith trace](https://smith.langchain.com/public/c3070198-5b13-419b-87bf-3821cdf34fa6/r), we can see what is going on under the hood:

* It's following what is explained in the [extraction](docs/use_cases/extraction).
* We call the `information_extraction` function on the input text.
* It will attempt to populate the provided schema from the url content.

## Research automation

We can automate the process of [web research](https://blog.langchain.dev/automating-web-research/) using a retriver.

![Image description](/img/web_research.png)

The `WebResearchRetriever` ([docs](https://python.langchain.com/docs/modules/data_connection/retrievers/web_research)).

Install requirments [here](https://github.com/langchain-ai/web-explorer/blob/main/requirements.txt):

`pip install -r requirements.txt`

Set `GOOGLE_CSE_ID` and `GOOGLE_API_KEY`.

In [40]:
import os
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.chat_models.openai import ChatOpenAI
from langchain.utilities import GoogleSearchAPIWrapper
from langchain.retrievers.web_research import WebResearchRetriever

In [41]:
# Vectorstore
vectorstore = Chroma(embedding_function=OpenAIEmbeddings(),persist_directory="./chroma_db_oai")

# LLM
llm = ChatOpenAI(temperature=0)

# Search 
search = GoogleSearchAPIWrapper()

Initialize retriever with the above tools to:
    
* Use an LLM to generate multiple relevant search queries (one LLM call)
* Execute a search for each query
* Choose the top K links per query  (multiple search calls in parallel)
* Load the information from all chosen links (scrape pages in parallel)
* Index those documents into a vectorstore
* Find the most relevant documents for each original generated search query

In [42]:
# Initialize
web_research_retriever = WebResearchRetriever.from_llm(
    vectorstore=vectorstore,
    llm=llm, 
    search=search)

In [44]:
# Run
import logging
logging.basicConfig()
logging.getLogger("langchain.retrievers.web_research").setLevel(logging.INFO)
from langchain.chains import RetrievalQAWithSourcesChain
user_input = "How do LLM Powered Autonomous Agents work?"
qa_chain = RetrievalQAWithSourcesChain.from_chain_type(llm,retriever=web_research_retriever)
result = qa_chain({"question": user_input})
result

INFO:langchain.retrievers.web_research:Generating questions for Google Search ...
INFO:langchain.retrievers.web_research:Questions for Google Search (raw): {'question': 'How do LLM Powered Autonomous Agents work?', 'text': LineList(lines=['1. What is the functioning principle of LLM Powered Autonomous Agents?\n', '2. How do LLM Powered Autonomous Agents operate?\n'])}
INFO:langchain.retrievers.web_research:Questions for Google Search: ['1. What is the functioning principle of LLM Powered Autonomous Agents?\n', '2. How do LLM Powered Autonomous Agents operate?\n']
INFO:langchain.retrievers.web_research:Searching for relevat urls ...
INFO:langchain.retrievers.web_research:Searching for relevat urls ...
INFO:langchain.retrievers.web_research:Search results: [{'title': 'LLM Powered Autonomous Agents | Hacker News', 'link': 'https://news.ycombinator.com/item?id=36488871', 'snippet': 'Jun 26, 2023 ... Exactly. A temperature of 0 means you always pick the highest probability token (i.e. the "

{'question': 'How do LLM Powered Autonomous Agents work?',
 'answer': "LLM-powered autonomous agents work by using LLM as the agent's brain, complemented by several key components such as planning, memory, and tool use. In terms of planning, the agent breaks down large tasks into smaller subgoals and can reflect and refine its actions based on past experiences. Memory is divided into short-term memory, which is used for in-context learning, and long-term memory, which allows the agent to retain and recall information over extended periods. Tool use involves the agent calling external APIs for additional information. These agents have been used in various applications, including scientific discovery and generative agents simulation.",
 'sources': ''}

### Going deeper 

* Here's a [app](https://github.com/langchain-ai/web-explorer/tree/main) that wraps this retriver with a lighweight UI.